In [1]:
import mlxtend
import numpy as np
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
import datetime as dt

date = dt.datetime.now()
if int(date.month)>7:
    semester=2
else:
    semester=1
    
SW=pd.read_excel('C:/Users/USer\Desktop/졸업작품/18-2 소웨 전공.xls')

student_ID=int(input('학번을 입력하세요 : '))
year=int(input('학년을 입력하세요 : '))
if year==3 or year==4 :
    track=str(input('어떤 트랙을 수강하고 계시는지 입력하세요(빅데이터/일반/IOT/보안) : '))
    
fav_pro=str(input('가장 선호하는 전공교수님을 입력하세요 : '))
hate_pro=str(input('가장 선호하지 않는 전공교수님을 입력하세요 : '))
fav_pro30=fav_pro+"30"
fav_pro20=fav_pro+"20"

hate_pro0=hate_pro+"0"
hate_pro_minus=hate_pro+'-'


data_df=pd.read_csv('C:/Users/USer/Desktop/졸업작품/졸작수강기록.csv')
with open('C:/Users/USer/Desktop/졸업작품/졸작수강기록.csv') as myfile:
    total_lines = sum(1 for line in myfile)
    
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)



# run example
if __name__ == "__main__":
    R = data_df.to_numpy()
    
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=False)
factorizer.fit()
result=factorizer.get_complete_matrix()

result_df=pd.DataFrame(result)
result_df.columns=["한명묵","정용주","최아영","민연아","강상우","이주형","노웅기","최재영","정옥란","황효석","이상웅","유준","최재혁","조정찬","김원","최기호","정윤현","민홍","오영민","차영운","김철연","전영철"]
result_df=(result_df*15).round(-1).astype(int)

result_df['한명묵']=result_df['한명묵'].map('한명묵{}'.format)
result_df['정용주']=result_df['정용주'].map('정용주{}'.format)
result_df['최아영']=result_df['최아영'].map('최아영{}'.format)
result_df['민연아']=result_df['민연아'].map('민연아{}'.format)
result_df['강상우']=result_df['강상우'].map('강상우{}'.format)

result_df['이주형']=result_df['이주형'].map('이주형{}'.format)
result_df['노웅기']=result_df['노웅기'].map('노웅기{}'.format)
result_df['최재영']=result_df['최재영'].map('최재영{}'.format)
result_df['정옥란']=result_df['정옥란'].map('정옥란{}'.format)
result_df['황효석']=result_df['황효석'].map('황효석{}'.format)

result_df['이상웅']=result_df['이상웅'].map('이상웅{}'.format)
result_df['유준']=result_df['유준'].map('유 준{}'.format)
result_df['최재혁']=result_df['최재혁'].map('최재혁{}'.format)
result_df['조정찬']=result_df['조정찬'].map('조정찬{}'.format)
result_df['김원']=result_df['김원'].map('김 원{}'.format)

result_df['최기호']=result_df['최기호'].map('최기호{}'.format)
result_df['정윤현']=result_df['정윤현'].map('정윤현{}'.format)
result_df['민홍']=result_df['민홍'].map('민 홍{}'.format)
result_df['오영민']=result_df['오영민'].map('오영민{}'.format)
result_df['차영운']=result_df['차영운'].map('차영운{}'.format)

result_df['김철연']=result_df['김철연'].map('김철연{}'.format)
result_df['전영철']=result_df['전영철'].map('전영철{}'.format)
result_df.to_csv('C:/Users/USer/Desktop/졸업작품/sgd.csv',header=False,index=False,encoding='utf-8-sig')
sgd_df=pd.read_csv('C:/Users/USer/Desktop/졸업작품/sgd.csv')
sgd_df.columns=["한명묵","정용주","최아영","민연아","강상우","이주형","노웅기","최재영","정옥란","황효석","이상웅","유준","최재혁","조정찬","김원","최기호","정윤현","민홍","오영민","차영운","김철연","전영철"]
sgd_np=sgd_df.to_numpy()
te = TransactionEncoder()
te_ary = te.fit(sgd_np).transform(sgd_np)
df = pd.DataFrame(te_ary, columns=te.columns_)

fp=fpgrowth(df, min_support=0.2,use_colnames=True)
result_fp=pd.DataFrame(fp)
result_fp.to_csv('C:/Users/USer/Desktop/졸업작품/fp.csv',header=False,index=False,encoding='utf-8-sig')

relation=pd.read_csv('C:/Users/USer/Desktop/졸업작품/fp.csv')

#relation.rename(columns={relation.columns[0]:"support", relation.columns[1]:'itemsets'},inplace=True)
relation.columns=['support','itemsets']

condition20=relation[relation['itemsets'].str.contains(fav_pro20)]
condition30=relation[relation['itemsets'].str.contains(fav_pro30)]
fav=pd.concat([condition20,condition30])
fav=fav.sort_values(by=['support'],axis=0,ascending=False)
SW_year=SW['수강학년']==year
search=SW[SW_year]
lecture=search[['교과명','교수성명','점수']]

# 피하고싶은 교수님을 데이터프레임에서 제거
fav = fav[~fav['itemsets'].str.contains(hate_pro, na=False, case=False)]
fav = fav[~fav['itemsets'].str.contains('-', na=False, case=False)]

# 학과에 안계신 교수님들 데이터프레임에서 제거
lecture = lecture[~lecture['교수성명'].str.contains('황효석', na=False, case=False)]
lecture = lecture[~lecture['교수성명'].str.contains('민연아', na=False, case=False)]

recommend=[]
for item in fav['itemsets']:
    if fav_pro in item:
        item = item.replace('0',"")
        item = item.replace('2',"")
        item = item.replace('3',"")
        item = item.replace('1',"")
        item = item.replace('4',"")
        item = item.replace(fav_pro,"")
        item=item.replace('frozenset({\'','')
        item=item.replace('\', \'\'})','')
        item=item.replace('\'})','')
        item=item.replace('\'','')
        item=item.replace(', ','')
        item=item.replace(' ','')
        recommend.append(item)

count=0
final=pd.DataFrame()
recommend=pd.DataFrame(recommend)
recommend.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
recommend.dropna(inplace=True)
recommend.columns=['교수성명']
copy=lecture.copy()
copy.drop_duplicates(['교과명'], keep='first', inplace=True, ignore_index=False)
num_prolec=len(copy)

# 선호하는 교수님이 있다면 그 교수님의 강의를 무조건 추천
final=pd.concat([final,lecture[lecture['교수성명'].str.contains(fav_pro)]])

for pro in recommend['교수성명']:
    for prof in lecture['교수성명']:
        if pro==prof and count!=num_prolec:
            final=pd.concat([final,lecture[lecture['교수성명'].str.contains(prof)]])
            count=count+1
            continue

final.drop_duplicates(['교과명'], keep='first', inplace=True, ignore_index=False)

for lec1 in final['교과명']:
    for lec2 in lecture['교과명']:
        if lec1==lec2 :
            lecture = lecture[~lecture['교과명'].str.contains(lec1, na=False, case=False)]
            
lecture=lecture.sort_values(by=['점수'],axis=0,ascending=False)
lecture.drop_duplicates(['교과명'], keep='first', inplace=True, ignore_index=False)
final=pd.concat([final,lecture])

print(recommend)
print(final)


학번을 입력하세요 : 18
학년을 입력하세요 : 1
가장 선호하는 전공교수님을 입력하세요 : 최재영
가장 선호하지 않는 전공교수님을 입력하세요 : 이상웅
           교수성명
0              
2           한명묵
3           정윤현
4           차영운
5            김원
7           최아영
8        정윤현차영운
9        정윤현한명묵
10        김원차영운
11        정윤현김원
12       한명묵차영운
13    정윤현차영운한명묵
14          정옥란
15          김철연
16        한명묵김원
17     정윤현김원차영운
18     정윤현김원한명묵
19          정용주
20       한명묵정윤현
22           민홍
23       정윤현최아영
24  정윤현김원한명묵차영운
25        정윤현민홍
26     한명묵김원차영운
27          최기호
28       김철연최아영
29          노웅기
30       정윤현김철연
31          오영민
32          조정찬
33    한명묵김철연정윤현
34       한명묵김철연
36    한명묵정윤현최아영
37        민홍최아영
38       차영운최아영
39       한명묵최아영
40       최기호강상우
41        차영운민홍
42          강상우
                     교과명                  교수성명    점수
31                  확률통계                   최아영  91.5
5                   로봇공학                   정용주  93.8
34  Practical English A2       Lawrence Minton  97.5
43  Practical English B2            Hannah Yoo  96.6
69  Pract